In [1]:
import os
import dotenv
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import json
import re
from tqdm import tqdm
from itertools import product
import umap
import hdbscan
import pickle
from sklearn.metrics import silhouette_score
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import torch
print("GPU is available:", torch.cuda.is_available())

from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import warnings
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
  print("Current device:", torch.cuda.get_device_name(0))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cwd = os.getcwd()
data_path = os.path.join(cwd,'data')
env_path = os.path.join(cwd,'.env')
models_path = os.path.join(cwd,'models')


GPU is available: True
Current device: NVIDIA GeForce RTX 3060


In [2]:
def evaluate_clustering(reduced_embeddings, labels, metrics=['silhouette', 'calinski', 'davies'],detail=False):
    """
    Evaluate clustering using multiple metrics.
    """
    scores = []
    unique_labels = set(labels)
    n_clusters = len([x for x in unique_labels if x != -1])
    
    if n_clusters < 2:
        return -np.inf
        
    try:
        if 'silhouette' in metrics:
            sil_score = silhouette_score(reduced_embeddings, labels)
            scores.append(sil_score)
            
        if 'calinski' in metrics:
            cal_score = calinski_harabasz_score(reduced_embeddings, labels)
            scores.append(np.log1p(cal_score) / 10)
            
        if 'davies' in metrics:
            dav_score = davies_bouldin_score(reduced_embeddings, labels)
            scores.append(1 / (1 + dav_score))

        if detail:
            return {'mean':np.mean(scores),
                    'silhouette':sil_score,
                    'calinski':cal_score,
                    'davies':dav_score,
                    'log_calinski':np.log1p(cal_score) / 10,
                    'inverse_davies':1 / (1 + dav_score),
                   }
        else:
            return np.mean(scores)
    except:
        return -np.inf

In [3]:
size = 15000

file_path = os.path.join(data_path,'final_reviews.csv')
df = pd.read_csv(file_path).dropna(subset='review')

good_reviews = df.loc[df['voted_up'] == True]['review'].drop_duplicates()
bad_reviews = df.loc[df['voted_up'] == False]['review'].drop_duplicates()

good_reviews = list(good_reviews.sample(size,random_state=42))
bad_reviews = list(bad_reviews.sample(size,random_state=42))

good_reviews = [str(i) for i in good_reviews]
bad_reviews = [str(i) for i in bad_reviews]

In [4]:
n = 5000
random_seed = 42

reviews_key = 'bad_reviews'
reviews_val = bad_reviews[:n]

embeddings = np.load(os.path.join(models_path, f'{reviews_key}_embeddings.npy'))[:n]

with open(os.path.join(models_path,f'{reviews_key}_best_params.json'), 'r') as file:
        best_params = json.load(file)

umap_reducer = umap.UMAP(**best_params.get('umap'), random_state=random_seed)
reduced_embeddings = umap_reducer.fit_transform(embeddings)

clusterer = hdbscan.HDBSCAN(**best_params.get('hdbscan'))
labels = clusterer.fit_predict(reduced_embeddings)

evaluate_clustering(reduced_embeddings, labels, detail=True)

{'mean': 0.541826167005016,
 'silhouette': 0.3770251,
 'calinski': 3449.7770808289783,
 'davies': 1.3051144935887657,
 'log_calinski': 0.8146354725482494,
 'inverse_davies': 0.4338179308582321}

In [5]:
df_optimized_general = pd.read_csv(os.path.join(models_path,f'{reviews_key}_optimized_llama_topics.csv'))
print(df_optimized_general.groupby('cluster').count())
df_optimized_general.groupby('cluster').sample(5,random_state=42).sort_values('cluster').head(50)

         text
cluster      
-1       1276
 0        906
 1       1367
 2       1451


,text,cluster
403,Not as good as bloodborne,-1
208,It runs like donkeybutt. Other than that it's just ok I guess. Dragon's dogma is way better.,-1
304,just get Runes to lose them. Pain,-1
2618,Screw it. Getting to a point where the stuttering just gets me killed. No word from no one on the subject. I'm just getting tired. Buy on sale or if they'll ever fix their console ports,-1
1748,Good game. Poor performance.,-1
4728,its an impossible game. you cannot play because you will meet a big boss and cannot pass. so those japans producers are mother fuckers idiots,0
25,would rather play modded ds3 than this,0
4982,dog ass compared to ds2,0
2645,got called maidenless and quit this game is shit,0
3186,the sup is bad,0


In [6]:
cluster = 1

df_optimized = pd.read_csv(os.path.join(models_path,f'{reviews_key}_optimized_llama_topics.csv'))

cluster_df = df_optimized.loc[df_optimized['cluster'] == cluster]

text_list = list(cluster_df['text'])
cluster_indexes = cluster_df.index

embeddings = np.load(os.path.join(models_path, f'{reviews_key}_embeddings.npy'))[:n]

embedding_indexes = embeddings[cluster_indexes]

with open(os.path.join(models_path,f'{reviews_key}_cluster_{cluster}_best_params.json'), 'r') as file:
        best_params = json.load(file)

umap_reducer = umap.UMAP(**best_params.get('umap'), random_state=random_seed)
reduced_embeddings = umap_reducer.fit_transform(embedding_indexes)

clusterer = hdbscan.HDBSCAN(**best_params.get('hdbscan'))
labels = clusterer.fit_predict(reduced_embeddings)

evaluate_clustering(reduced_embeddings, labels, detail=True)

{'mean': 0.47390394874487285,
 'silhouette': 0.30148986,
 'calinski': 640.6993905771029,
 'davies': 1.110550683154865,
 'log_calinski': 0.6464119955055384,
 'inverse_davies': 0.4738099909096679}

In [7]:
df_optimized_cluster = pd.read_csv(os.path.join(models_path,f'{reviews_key}_cluster_{cluster}_optimized_llama_topics.csv'))
print(df_optimized_cluster.groupby('cluster').count())
df_optimized_cluster.groupby('cluster').sample(5,random_state=42).sort_values('cluster').head(50)

         text
cluster      
-1        572
 0        236
 1        559


text  \
1227                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    